# MDの.posファイルからH2Oの座標を取得して，それを出力ファイルに書き出す．

In [19]:
import numpy as np
import shutil
import os

In [20]:
# 水分子数の指定( modify here)
NUM_MOLECULE=1
print("水分子の数：：", NUM_MOLECULE)

# pseudo_dirの指定( modify here)
PSEUDO_DIR=" '/home/amano/src/qe-6.4.1/pseudo/' "
# pseudo_potentialの指定( modify here)
PSEUDO_H="H.pbe-rrkjus_psl.1.0.0.UPF"
PSEUDO_O="O.pbe-n-rrkjus_psl.1.0.0.UPF"
#PSEUDO_O="O.blyp-mt.UPF"
#PSEUDO_H="H.blyp-vbc.UPF"

水分子の数：： 1


In [21]:
## MDのstep数目の座標を用いて，scf.inputとwan.inputで使用する座標の列を作成する．
## posファイルのOとHの順番によってここを変更する必要あり．(OHHOHH...を想定している．)
def make_cordinate_list(zahixyou, NUM_MOLECULE, step):
    line=[]
    for water_num in np.arange(NUM_MOLECULE) : 
        # 3つの座標の取得
        O=zahixyou[3*NUM_MOLECULE*step+water_num*3]
        H1=zahixyou[3*NUM_MOLECULE*step+water_num*3+1]
        H2=zahixyou[3*NUM_MOLECULE*step+water_num*3+2]
        tmp=[
            " O {0} {1} {2}".format(O[0], O[1], O[2]), 
            " H {0} {1} {2}".format(H1[0], H1[1], H1[2]), 
            " H {0} {1} {2}".format(H2[0], H2[1], H2[2]), 
        ]
        line=line+tmp
    return line

In [22]:
## scfのinputファイルを作成( modify here)
#O，H1，H2はいずれも3次元の座標
def make_scffile(zahixyou,NUM_MOLECULE, step):
    lines1= [
    "&control", 
    "      calculation  = 'scf'", 
    "      prefix         = 'h2o_bec'", 
    "      pseudo_dir = {0}".format(PSEUDO_DIR), 
    "      outdir         = './work/'", 
    " ",    
    "/", 
    "&system", 
    "      ibrav=1",
    "      celldm(1) =10", 
    "      nat={0},".format(3*NUM_MOLECULE),
    "      ntyp=2,", 
    "      ecutwfc=80.0", 
    "      nosym =.TRUE.", 
    "/", 
    "&electrons", 
    "      conv_thr=1e-8", 
    "/", 
    "&ions", 
    "/", 
    "ATOMIC_SPECIES", 
    "H   1.00d0 {0}".format(PSEUDO_H), 
    "O  16.0d0 {0}".format(PSEUDO_O), 
    " ",
    " " ,  
    "ATOMIC_POSITIONS (angstrom)", 
    ]
    # 座標部分
    lines2=make_cordinate_list(zahixyou, NUM_MOLECULE, step)
    
    lines3=[
    " " , 
    "K_POINTS (automatic)", 
    " 1 1 1 0 0 0", 
    ]
    
    lines=lines1+lines2+lines3
    
    #出力ファイル名
    output="h2o_bec.pw.in"
    #ファイルの保存
    with open(output, mode='w') as f:
        f.write('\n'.join(lines))


In [23]:
# wannier90のインプットファイルの作成( modify here)
# ここで結晶構造も必要なので場合によって書き換えること！！
def make_wanfile(zahixyou,NUM_MOLECULE, step):
        lines1=[
            "# num_wann sould be equal to nbnd",
            "# default nbnd in insulater is electron num/2",
            "num_wann        = {0}".format(NUM_MOLECULE*4),
            "num_iter        = 100",
            " ",
            "translate_home_cell=true",
            " ",
            "guiding_centres = true",
            " ",
            "# level of verbosity of the output from 0 to 3(high)",
            "iprint          = 2",
            " ",
            "wannier_plot=.true.",
            "# wannier function output format",
            "wannier_plot_format = cube",
            "wannier_plot_mode = molecule",
            " ",
            " ",
            "begin atoms_cart",
            "ang",
        ]
        # 座標部分
        lines2=make_cordinate_list(zahixyou, NUM_MOLECULE, step)
        
        lines3=[
            "end atoms_cart",
            " ",
            "begin projections",
            "random",
            "end projections",
            " ",
            "# ang or bohr",
            "begin unit_cell_cart",
            "bohr",
            "10.00000000000   0.00000000000000 0.00000000000000",
            " 0.00000000000000 10.0000000000000 0.00000000000000",
            " 0.00000000000000 0.00000000000000 10.0000000000000",
            "end unit_cell_cart",
            " ",
            "mp_grid    : 1 1 1",
            "gamma_only : true",
            " ",
            "begin kpoints",
            "0.0 0.0 0.0",
            "end kpoints",
        ]
        lines=lines1+lines2+lines3
        
        #出力ファイル名
        output="h2o_bec.win"
        #ファイルの保存
        with open(output, mode='w') as f:
            f.write('\n'.join(lines))


In [24]:
# phファイルの作成
# 計算が重いようなら，zeu=trueにして，zueとasrの行を削除
# tr2_phも10d-14で大丈夫かも
def make_phfile():
    lines=[
    "h2o phonon",
    "&inputph",
    "     tr2_ph = 1.0d-15," ,
    "     prefix = 'h2o_bec'" ,
    "     outdir = './work/'"  ,
    "     trans  = .true.", 
    "     zue    = .true.", 
    "     epsil  = .true.", 
    "     asr      = .true.",    
    "/",
    "0 0 0",
    ]
    #出力ファイル名
    output="h2o_bec.ph.in"
    #ファイルの保存
    with open(output, mode='w') as f:
        f.write('\n'.join(lines))


In [25]:
# スクリプトファイルを作成（modify here）
#copy.shの作成
def make_scriptfile():
    lines=[
    "# scf ",
    "mpirun -np 24 pw.x < h2o_bec.pw.in > h2o_bec.pw.out 2>&1",
    "# ph ",
    "mpirun -np 24 ph.x < h2o_bec.pw.in > h2o_bec.pw.out 2>&1",
    "# pre wannier90",
    "mpirun -np 24  wannier90.x -pp h2o_bec 2>&1",
    "# pw2wan",
    "mpirun -np 24  pw2wannier90.x < h2o_bec.pw2wan > pw2wan.out 2>&1",
    "# wannier90",
    "mpirun -np 24  wannier90.x h2o_bec 2>&1",
    ]
    #出力ファイル名
    output="script.sh"
    #ファイルの保存
    with open(output, mode='w') as f:
        f.write('\n'.join(lines))


In [26]:
# スクリプトの実行用ファイル
#slurm.shの作成
def make_jobfile(SIZE):
    lines=[
    "#!/bin/sh",
    "number=0",
    " ",
    "# 現在のdirを取得",
    "CURRENT_DIR=$(pwd)",
    " ",
    "while test $number -le {0}".format(SIZE),
    "do",
    " ",
    "    #working dirへ移動",
    "    cd ${CURRENT_DIR}/water_$number/",
    " ",
    "    #jobファイルの実行",  # modify here
    "    sbatch sample4.slurm",
    " ",
    "    number=`expr $number + 1`",
    "done",
    ]
    #出力ファイル名
    output="job.sh"
    #ファイルの保存
    with open(output, mode='w') as f:
        f.write('\n'.join(lines))



In [27]:
# pw2wanファイルの作成
def make_pw2wanfile():
    lines=[
   "&inputpp",
   "  outdir         = './work/' ",
   "  prefix         = 'h2o_bec' ",
   "  seedname       = 'h2o_bec' ",
   "  spin_component = 'none' ",
   "  write_mmn      = .true. ",
   "  write_amn      = .true. ",
   "  write_unk      = .true. " ,
   "/ ",
    ]
    #出力ファイル名
    output="h2o_bec.pw2wan"
    #ファイルの保存
    with open(output, mode='w') as f:
        f.write('\n'.join(lines))


In [28]:
#filenameファイルのディレクトリを移動する．
# shutil.copyfileはscrのファイルをcopyにコピーする．
def copyfile1(num,filename):
    import shutil
    copy = './water_{}'.format(str(num)) #移動先のdir
    shutil.move(filename,copy)

#water_10などのdirを作成
def make_dir(new_dir_path):
    import os
    import shutil
    if os.path.exists(new_dir_path)==True:
        shutil.rmtree(new_dir_path) # すでに存在すれば削除
    os.mkdir(new_dir_path)

In [29]:
# 実際にファイル群を作成する関数
def make_files(zahixyou,SIZE,NUM_MOLECULE):
    for step in np.arange(SIZE):
        print("ステップ：：", step)

        #water_stepのdir作成
        make_dir("water_{0}".format(step))
    
        #必要ファイル作成
        make_scffile(zahixyou, NUM_MOLECULE, step)
        make_phfile()
        make_wanfile(zahixyou, NUM_MOLECULE, step)
        make_pw2wanfile()
        make_scriptfile()
    
        #ファイル移動
        copyfile1(step,"h2o_bec.pw.in")
        copyfile1(step,"h2o_bec.pw2wan")
        copyfile1(step,"h2o_bec.ph.in")
        copyfile1(step,"h2o_bec.win")
        copyfile1(step,"script.sh")

    # 最後にjobファイルの作成
    make_jobfile(SIZE)

In [30]:
# 結果のwanデータの処理をするスクリプトファイル作成（wan.sh）
def make_wan_extract(SIZE, NUM_MOLECULE):
    lines=[
    "#!/bin/sh",
    "number=0",
    " ",
    "while test $number -le {0}".format(SIZE),
    "do",
    "    #データの抽出",
    "    grep -A {0} 'Final State' ./water_$number/h2o_bec.wout > ./water_$number/wan.txt".format(4*NUM_MOLECULE),
    "    #余計な行を削除",
    "    sed -i '1d' ./water_$number/wan.txt",
    "    #データの抽出",
    "    awk '{ print $7,$8,$9 }' ./water_$number/wan.txt > ./result/wan2_$number.txt",
    "    sed -i -e 's/,/ /g' ./result/wan2_$number.txt",
    "     "
    "    number=`expr $number + 1`",
    "done",
    ]
    #出力ファイル名
    output="wan.sh"
    #ファイルの保存
    with open(output, mode='w') as f:
        f.write('\n'.join(lines))




In [31]:
# 結果のBECデータを処理するスクリプトファイル作成（bec.sh）
def make_becfile(SIZE, NUM_MOLECULE):
    lines=[
    "#!/bin/sh",
    "number=0",
    " ",
    "while test $number -le {0}".format(SIZE),
    "do",
    "    #データの抽出",
    "    # zueかzeuかで変わる点にも注意．(modify here)",
    "    #grep -A {0} ' Effective charges (d Force / dE) in cartesian axis' ./water_$number/h2o_bec.wout > ./water_$number/wan.txt".format(4*NUM_MOLECULE+2),        
    "    grep -A {0} ' Effective charges (d P / du) in cartesian axis' ./water_$number/h2o_bec.wout > ./water_$number/wan.txt".format(4*NUM_MOLECULE+2),
    "    #余計な行を削除",
    "    sed -i '1,2d' ./water_$number/bec.txt",
    "    sed -i '/atom/d' ./water_$number/bec.txt",
    "    #データの抽出",
    "    awk '{ print $3,$4,$5 }' ./water_$number/bec.txt > ./result/bec_$number.txt"
    "     "
    "    number=`expr $number + 1`",
    "done",
    ]
    #出力ファイル名
    output="bec.sh"
    #ファイルの保存
    with open(output, mode='w') as f:
        f.write('\n'.join(lines))


In [32]:
#

#make_wan_extract(SIZE, NUM_MOLECULE)
#make_becfile(SIZE, NUM_MOLECULE)

NameError: name 'SIZE' is not defined

# 自分で座標リストを作る場合はここを使用する．

relaxした構造は
#rはÅで指定
r=0.9693
Angle is 104.107 deg



In [36]:
#
r=0.9693
angle=104.107*np.pi/180

# np.arrayは必要な角度*3つ．
cord_list=np.zeros([360*3,3])
#print(cord_list)

for i in range(360):
        cord_list[3*i]=np.array([0,0,0]) #O
        cord_list[3*i+1]=np.array([r,0,0])                     #H1
        cord_list[3*i+2]=np.array([r*np.cos(angle),r*np.sin(angle)*np.cos(i*np.pi/180),r*np.sin(angle)*np.sin(i*np.pi/180)])                    #H2


print(cord_list)

[[ 0.          0.          0.        ]
 [ 0.9693      0.          0.        ]
 [-0.23625088  0.94006809  0.        ]
 ...
 [ 0.          0.          0.        ]
 [ 0.9693      0.          0.        ]
 [-0.23625088  0.93992491 -0.01640645]]


In [37]:
### メイン関数
def main():
    import subprocess
    # # md結果のposファイルを読み込み．
    #zahixyou=np.loadtxt("h2o_mol2.pos")
    #print(zahixyou)
    # zahixyou=cord_list
    
    # 出力されているMDステップの数
    SIZE=int(cord_list.shape[0]/(NUM_MOLECULE*3))
    print("ステップ数：：", SIZE)
    
    # ジョブファイル作成
    make_files(cord_list,SIZE,NUM_MOLECULE)

    # ジョブの実行
    #subprocess.run(['./job.sh'])
    
    
    # 計算終了後，becとwanのデータを収集
    # これでresult以下にデータが収集される．
    make_dir("result")
    make_becfile(SIZE, NUM_MOLECULE)
    make_wan_extract(SIZE, NUM_MOLECULE)
    #subprocess.run(['./bec.sh'])
    #subprocess.run(['./wan.sh'])

In [38]:
main()

ステップ数：： 360
ステップ：： 0
ステップ：： 1
ステップ：： 2
ステップ：： 3
ステップ：： 4
ステップ：： 5
ステップ：： 6
ステップ：： 7
ステップ：： 8
ステップ：： 9
ステップ：： 10
ステップ：： 11
ステップ：： 12
ステップ：： 13
ステップ：： 14
ステップ：： 15
ステップ：： 16
ステップ：： 17
ステップ：： 18
ステップ：： 19
ステップ：： 20
ステップ：： 21
ステップ：： 22
ステップ：： 23
ステップ：： 24
ステップ：： 25
ステップ：： 26
ステップ：： 27
ステップ：： 28
ステップ：： 29
ステップ：： 30
ステップ：： 31
ステップ：： 32
ステップ：： 33
ステップ：： 34
ステップ：： 35
ステップ：： 36
ステップ：： 37
ステップ：： 38
ステップ：： 39
ステップ：： 40
ステップ：： 41
ステップ：： 42
ステップ：： 43
ステップ：： 44
ステップ：： 45
ステップ：： 46
ステップ：： 47
ステップ：： 48
ステップ：： 49
ステップ：： 50
ステップ：： 51
ステップ：： 52
ステップ：： 53
ステップ：： 54
ステップ：： 55
ステップ：： 56
ステップ：： 57
ステップ：： 58
ステップ：： 59
ステップ：： 60
ステップ：： 61
ステップ：： 62
ステップ：： 63
ステップ：： 64
ステップ：： 65
ステップ：： 66
ステップ：： 67
ステップ：： 68
ステップ：： 69
ステップ：： 70
ステップ：： 71
ステップ：： 72
ステップ：： 73
ステップ：： 74
ステップ：： 75
ステップ：： 76
ステップ：： 77
ステップ：： 78
ステップ：： 79
ステップ：： 80
ステップ：： 81
ステップ：： 82
ステップ：： 83
ステップ：： 84
ステップ：： 85
ステップ：： 86
ステップ：： 87
ステップ：： 88
ステップ：： 89
ステップ：： 90
ステップ：： 91
ステップ：： 92
ステップ：： 93
ステップ：： 94
ステップ：： 95
ステップ：： 96
ステップ：： 97
ステップ：： 98
ステップ：： 9

In [121]:
###ワニエセンターから双極子モーメントを計算する例
# 双極子モーメントベクトルの保存．
pol_wan_list=np.zeros([SIZE,3])

for i in range(0,SIZE):  
    #初期化
    tmp_polarization=0
    #水分子の座標の獲得
    for water_num in np.arange(NUM_MOLECULE) : 
        O=zahixyou[3*NUM_MOLECULE*step+water_num*3]
        H1=zahixyou[3*NUM_MOLECULE*step+water_num*3+1]
        H2=zahixyou[3*NUM_MOLECULE*step+water_num*3+2]
        #双極子モーメントに加算
        tmp_polarization+=6*O+H1+H2

    #WCデータをファイルから読み込み            
    WCs=np.loadtxt("./result/wan2_{}.txt".format(i))

    for j in range(0,4*NUM_MOLECULE):
        #双極子モーメントに加算
        tmp_polariaztion+=-WCs[j]
    
    print(i, tmp_polarization)
    pol_wan_list[i]=tmp_polarization


In [ ]:
## BECから双極子モーメントの差を計算する例
# zeuの場合，追加でAcoustic sum ruleの実装が必要．

# polの差の保存．（初期状態との差）
pol_bec_list=np.zeros([SIZE,3])

# 初期化
pol=0

for i in range(0,SIZE-1):
    # 有効電荷の取得 
    Zs=np.loadtxt("./result/bec_{}.txt".format(i))
    
    for j in range(0, 3*NUM_MOLECULE):
        tmp=3*i*NUM_MOLECULE+3*j
        pol+=np.dot(Zs[tmp:tmp+3],zahixyou[tmp+3]-zahixyou[tmp]) #O
        pol+=np.dot(Zs[tmp+3:tmp+6],zahixyou[tmp+4]-zahixyou[tmp+1]) #H1
        pol+=np.dot(Zs[tmp+6:tmp+9],zahixyou[tmp+5]-zahixyou[tmp+2]) #H2
    
    print(i, pol)    
    pol_list[i]=pol


In [ ]:
#回転ベクトル

###ワニエセンターから双極子モーメントを計算する例
# 双極子モーメントベクトルの保存．
pol_wan_list=np.zeros([SIZE,3])

for i in range(0,SIZE):  
    #初期化
    tmp_polarization=0

    kaiten=np.array([[[0, 0, 0],
                      [0, cos(i*np.pi/180)-1, -sin(i*np.pi/180)],
                      [0, sin(i*np.pi/180), cos(i*np.pi/180)-1]]])
                      

    #WCデータをファイルから読み込み            
    WCs=np.loadtxt("./result/wan2_{}.txt".format(i))

    for j in range(0,4*NUM_MOLECULE):
        #双極子モーメントに加算
        tmp_polariaztion+=-WCs[j]
    
    print(i, tmp_polarization)
    pol_wan_list[i]=tmp_polarization



kaiten=np.array([[])